# AG2 Tool Example
<img src="https://raw.githubusercontent.com/AgentOps-AI/agentops/main/docs/images/external/autogen/autogen-integration.png?raw=true" width="25%"/>

To get started, you'll need to install the AgentOps package and [set an API key](app.agentops.ai).

AgentOps automatically configures itself when it's initialized meaning your agent run data will be tracked and logged to your AgentOps account right away.

First let's install the required packages

In [ ]:
%pip install -U pyautogen
%pip install -U agentops
%pip install -U python-dotenv

Then import them

In [1]:
from typing import Annotated, Literal
from autogen import ConversableAgent, register_function
import agentops
import os
from dotenv import load_dotenv
from IPython.core.error import (
    StdinNotImplementedError,
)  # only needed by AgentOps testing automation


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_insta

AttributeError: _ARRAY_API not found

Next, we'll set our API keys. There are several ways to do this, the code below is just the most foolproof way for the purposes of this notebook. It accounts for both users who use environment variables and those who just want to set the API Key here in this notebook.

[Get an AgentOps API key](https://agentops.ai/settings/projects)

1. Create an environment variable in a .env file or other method. By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Or...

2. Replace `<your_agentops_key>` below and pass in the optional `api_key` parameter to the AgentOps `init(api_key=...)` function. Remember not to commit your API key to a public repo!

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

In [3]:
agentops.init(AGENTOPS_API_KEY, default_tags=["autogen-tool-example"])

print("AgentOps is now running. You can view your session in the link above")

(DEBUG) 🖇 AgentOps: Including project_id in resource attributes: fd6006a1-e230-4416-b1c7-f442d195a718
(DEBUG) 🖇 AgentOps: Tracing core initialized
(DEBUG) 🖇 AgentOps: Instrumented OpenAIInstrumentor
(DEBUG) 🖇 AgentOps: Instrumented AnthropicInstrumentor
(DEBUG) 🖇 AgentOps: Instrumented CohereInstrumentor
(DEBUG) 🖇 AgentOps: [opentelemetry.trace] Overriding of current TracerProvider is not allowed
(DEBUG) 🖇 AgentOps: Instrumented CrewAIInstrumentor
(DEBUG) 🖇 AgentOps: Instrumented GroqInstrumentor
(DEBUG) 🖇 AgentOps: [opentelemetry.instrumentation.instrumentor] DependencyConflict: requested: "haystack-ai >= 2.0.0" but found: "haystack-ai 2.11.0rc0"
(DEBUG) 🖇 AgentOps: Instrumented HaystackInstrumentor
(DEBUG) 🖇 AgentOps: Instrumented MistralAiInstrumentor
(DEBUG) 🖇 AgentOps: Package ollama not found; skipping instrumentation of OllamaInstrumentor
(DEBUG) 🖇 AgentOps: Instrumented AgentsInstrumentor
(DEBUG) 🖇 AgentOps: [opentelemetry.instrumentation.autogen.instrumentation] Instrumenting 

AgentOps is now running. You can view your session in the link above


AG2 will now start automatically tracking

* LLM prompts and completions
* Token usage and costs
* Agent names and actions
* Correspondence between agents
* Tool usage
* Errors

# Tool Example
AgentOps tracks when AG2 agents use tools. You can find more information on this example in [tool-use.ipynb](https://docs.ag2.ai/docs/tutorial/tool-use#tool-use)

In [4]:
# Define model, openai api key, tags, etc in the agent configuration
config_list = [
    {
        "model": "gpt-4-turbo",
        "api_key": OPENAI_API_KEY,
        "tags": ["mathagent-example", "tool"],
    }
]

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")


# Create the agent that uses the LLM.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(
    calculator
)
user_proxy.register_for_execution(name="calculator")(calculator)

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Let the assistant start the conversation.  It will end when the user types "exit".
try:
    user_proxy.initiate_chat(
        assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?"
    )
except StdinNotImplementedError:
    # This is only necessary for AgentOps testing automation which is headless and will not have user input
    print("Stdin not implemented. Skipping initiate_chat")
    agentops.end_session("Indeterminate")

agentops.end_session("Success")

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen\agentchat\conversable_agent.py:2996: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen\agentchat\conversable_agent.py:2905: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_200j96eatOaZnTf7FmD46Cti): calculator *****
Arguments: 
{"a": 1423, "b": 123, "operator": "-"}
***************************************************************************
***** Suggested tool call (call_ezUIWB1TrSKVuqLvV9h4N1Hh): calculator *****
Arguments: 
{"a": 32, "b": 23, "operator": "+"}
***************************************************************************

--------------------------------------------------------------------------------


(DEBUG) 🖇 AgentOps: [opentelemetry.attributes] Invalid type dict for attribute 'tool.name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types



>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_200j96eatOaZnTf7FmD46Cti
Input arguments: {'a': 1423, 'b': 123, 'operator': '-'}


(DEBUG) 🖇 AgentOps: [opentelemetry.attributes] Invalid type dict for attribute 'tool.name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


...


>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_C35g3JHTHjb9do7MqqGvoNin
Input arguments: {'a': 1300, 'b': 3, 'operator': '/'}


(DEBUG) 🖇 AgentOps: [opentelemetry.attributes] Invalid type dict for attribute 'tool.name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types



>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_WH1jvrIQWgiYOfu1XnqGOnoY
Input arguments: {'a': 55, 'b': 5, 'operator': '*'}
User (to Assistant):

***** Response from calling tool (call_C35g3JHTHjb9do7MqqGvoNin) *****
433
**********************************************************************

--------------------------------------------------------------------------------
***** Response from calling tool (call_WH1jvrIQWgiYOfu1XnqGOnoY) *****
275
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_zw2MlYqxdjNSyVFlP1IgntY6): calculator *****
Arguments: 
{"a":433,"b":275,"operator":"+"}
***************************************************************************

--------------------------------------------------------------------------------


(DEBUG) 🖇 AgentOps: [opentelemetry.attributes] Invalid type dict for attribute 'tool.name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types



>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: call_zw2MlYqxdjNSyVFlP1IgntY6
Input arguments: {'a': 433, 'b': 275, 'operator': '+'}
User (to Assistant):

***** Response from calling tool (call_zw2MlYqxdjNSyVFlP1IgntY6) *****
708
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

The result of the calculation (1423 - 123) / 3 + (32 + 23) * 5 is 708.

TERMINATE

--------------------------------------------------------------------------------


You can see your run in action at [app.agentops.ai](app.agentops.ai). In this example, the AgentOps dashboard will show:

* Agents talking to each other
* Each use of the `calculator` tool
* Each call to OpenAI for LLM use